## Gaussian Mixture Model

この例題は、以下のサイトで紹介されているコードを参考にしています。

- https://docs.pymc.io/notebooks/gaussian_mixture_model.html



## Install Package

In [ ]:
!pip install numpyro

【重要】パッケージのインストール完了後に、ランタイムを再起動して下さい！

## Import Package

In [ ]:
import numpyro
import numpyro.distributions as dist

import arviz as az

import jax
import jax.numpy as jnp

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.rcParams['font.size'] = 14

In [ ]:
numpyro.set_platform('cpu')
numpyro.set_host_device_count(4)

## Generate Data

In [ ]:
mu = [1, 5, 9]
sd = [1, 1, 1]

x0 = np.random.normal(mu[0], sd[0], 100)
x1 = np.random.normal(mu[1], sd[1], 100)
x2 = np.random.normal(mu[2], sd[2], 100)

In [ ]:
sns.kdeplot(x0, color='C0')
sns.kdeplot(x1, color='C1')
sns.kdeplot(x2, color='C2');

In [ ]:
x_obs = np.concatenate([x0, x1, x2])

## Define Model & Inference

In [ ]:
x_obs = jnp.array(x_obs)

In [ ]:
def model(K, x_obs):
    
    N = len(x_obs)
    
    p = numpyro.sample('p', dist.Dirichlet(jnp.ones(K)))
    
    # クラスターの大きさに関する条件
    numpyro.factor('cond_cluster_size', jnp.where(jnp.min(p) > 0.1, 0, -jnp.inf))
    
    k = numpyro.sample('cluster_index', dist.Categorical(p), sample_shape=(N,))
    
    mu = numpyro.sample('mu', dist.Normal(0, 10), sample_shape=(K,))
    sd = numpyro.sample('sd', dist.HalfNormal(5), sample_shape=(K,))
    
    # クラスターの順序に関する条件
    order = jnp.all(jnp.diff(mu) > 0)
    numpyro.factor('cond_cluster_order', jnp.where(order, 0, -jnp.inf))
    
    numpyro.sample('obs', dist.Normal(mu[k], sd[k]), obs=x_obs)

In [ ]:
nuts = numpyro.infer.NUTS(model, target_accept_prob=0.95)
kernel = numpyro.infer.DiscreteHMCGibbs(nuts)

mcmc = numpyro.infer.MCMC(kernel, num_warmup=5000, num_samples=1000, num_chains=4)

mcmc.run(jax.random.PRNGKey(0), 3, x_obs)
trace = mcmc.get_samples()

idata = az.from_numpyro(mcmc)

In [ ]:
az.plot_trace(idata, var_names=['mu', 'p', 'sd'])
plt.gcf().subplots_adjust(wspace=0.5, hspace=0.5)

In [ ]:
az.summary(idata, var_names=['mu', 'p', 'sd'])

## Check Latent Variable

In [ ]:
fig = plt.figure(figsize=(10, 6))

sns.heatmap(trace['cluster_index'], cmap='jet')

plt.title('Latent Variables');

In [ ]:
fig = plt.figure(figsize=(8, 4))

plt.plot(trace['cluster_index'].mean(axis=0))

plt.title('Mean Value of Latent Variables');